# Preparation of TReX dataset

In [11]:
# build a rdf dictionary from wikidata truthy
import json

entity_dictionary = {}
with open("/data/wikidata/latest-truthy.nt", "r") as f:
    for line in f:
        if "rdf-schema#label" in line and "@en " in line:
            try:
                s,p ,o = line.split("> ")
                s = s.replace("<", "")
                o = o.replace("@en", "")
                start = o.index( "\"" ) + 1
                end = o.index( "\"", start )
                o = o[start:end]
                s = s.encode('utf-8').decode('unicode-escape')
                o = o.encode('utf-8').decode('unicode-escape')
                    
                s = s.replace("http://www.wikidata.org/entity/","")
                o = o.replace("http://www.wikidata.org/entity/","")
                
                entity_dictionary[s] = o
            except ValueError:
                continue
#with open('data/data/entity_dictionary', 'w') as fp:
#    json.dump(entity_dictionary, fp, indent=4, sort_keys=True)
        

KeyboardInterrupt: 

In [12]:
print("test")

test


In [1]:
#Helper

def remove_duplicates(prop_triples):
    for p_qid in prop_triples:
        triples = prop_triples[p_qid]
        seen = set()
        new_l = []
        for d in triples:
            t = tuple(d.items())
            if t not in seen:
                seen.add(t)
                new_l.append(d)
        prop_triples[p_qid] = new_l
    return prop_triples

def substract_triples(original_prop_triples, substract_prop_triples):
    for p_qid in original_prop_triples:
        original_triples = original_prop_triples[p_qid]
        substract_triples = substract_prop_triples[p_qid]
        
        to_be_removed = set()
        for d in substract_triples:
            to_be_removed.add(tuple(d.items()))

        new_l = []
        for d in original_triples:
            t = tuple(d.items())
            if t not in to_be_removed:
                new_l.append(d)
        original_prop_triples[p_qid] = new_l
    return original_prop_triples

In [27]:
import json, os

trex_folder = "/home/kalo/conferences/akbc2021/data/data/TREx/"
entity_dictionary_file = "/home/kalo/conferences/akbc2021/data/data/entity_dictionary"

common_vocab_file = "/home/kalo/conferences/akbc2021/common_vocab_cased.txt"

#trex_triple_file = "/home/kalo/conferences/akbc2021/data/"+"trex_test.json"
trex_triple_file = "/data/fichtel/BERTriple/test_datasets/"+"LAMA_trex_test.json"
entity_dictionary_trex_label_file = "/data/fichtel/BERTriple/entity2label_trexlabel.json"
trex_test = {}


def get_label(uri,entity_dictionary):
    return entity_dictionary[uri] 

def load_trex_file(trex_data, entity_dictionary, common_vocab):
    global trex_test
    counter = 0
    for triple_dict in trex_data:
        s_qid = triple_dict["sub_uri"]
        p_qid = triple_dict["predicate_id"]
        o_qid = triple_dict["obj_uri"]
        s_label_trex = triple_dict["sub_label"]
        o_label_trex = triple_dict["obj_label"]
        if s_qid in entity_dictionary:
            s_label = entity_dictionary[s_qid]
            #change s_label to TREX label to have the same labels of testset and trainset
            if s_label_trex != s_label:
                entity_dictionary[s_qid] = s_label_trex
                
            if o_qid in entity_dictionary:
                o_label = entity_dictionary[o_qid]
                #change o_label to TREX label to have the same labels of testset and trainset
                if o_label_trex != o_label:
                    entity_dictionary[o_qid] = o_label_trex

                #add tripel with current labels if the o_label is in common_vocab to have obj_queries
                s_label = entity_dictionary[s_qid]
                o_label = entity_dictionary[o_qid]
                if o_label in common_vocab:
                    triple = {"subj": s_label, "prop": p_qid, "obj": o_label}
                    counter += 1
                    if p_qid not in trex_test:
                        trex_test[p_qid] = [triple]
                    else:
                        trex_test[p_qid].append(triple)
            else:
                print("no label found in wikidata:(", o_qid, o_label_trex)
        else:
            print("no label found wikidata :(", s_qid, s_label_trex)
    print("Added {} triples of property {}.".format(counter, p_qid))

with open(entity_dictionary_file, "r") as f:
    entity_dictionary = json.load(f)
    with open(common_vocab_file, "r") as f:
        common_vocab = set()
        for line in f:
            common_vocab.add(line.strip())
        for file in os.listdir(trex_folder):
            if file != "wikidata_training.json":
                with open(trex_folder+file, "r") as f:
                    trex_data = []
                    for line in f.readlines():
                        trex_data.append(json.loads(line))
                    load_trex_file(trex_data, entity_dictionary, common_vocab)
        
        count_all_triples = 0
        for p_qid in trex_test:
            for triple in trex_test[p_qid]:
                count_all_triples = count_all_triples + 1
        print("Lengths before removal {}.".format(count_all_triples))
              
        #remove duplicates of triples
        trex_test = remove_duplicates(trex_test)
        count_all_triples = 0
        for p_qid in trex_test:
            for triple in trex_test[p_qid]:
                count_all_triples = count_all_triples + 1
        print("Lengths after removal {}.".format(count_all_triples))
        
        #save the test dataset of LAMA
        with open(trex_triple_file, "w") as f:
            json.dump(trex_test,f, indent=4, sort_keys=True)

        #save the dictionary where the wikidata rdf labels are replaced with the trex labels when they were different
        with open(entity_dictionary_trex_label_file, 'w') as fp:
            json.dump(entity_dictionary, fp, indent=4, sort_keys=True)

no label found in wikidata:( Q3292203 interchange
no label found wikidata :( Q7579156 Spoon sweets
no label found wikidata :( Q412609 uridine 5'-triphosphoric acid
no label found wikidata :( Q7259490 pan loaf
no label found wikidata :( Q421647 Triuranium octoxide
no label found wikidata :( Q17004641 Episcopal lineage
no label found wikidata :( Q1323442 scorched rice
no label found wikidata :( Q7233414 Posset
no label found wikidata :( Q423223 L-lyxose
no label found wikidata :( Q28775 nicotinamide adenine dinucleotide
no label found wikidata :( Q28775 nicotinamide adenine dinucleotide
no label found wikidata :( Q7233414 Posset
Added 964 triples of property P527.
no label found wikidata :( Q18913178 Panagiotis Lafazanis
Added 976 triples of property P103.
no label found wikidata :( Q422281 Cholecystokinin
no label found wikidata :( Q1024108 COPI
no label found wikidata :( Q12745086 Inhibitor of apoptosis domain
no label found wikidata :( Q5324056 ERM protein family
no label found wikida

# get Wikidata training data



In [6]:
import json

training_data = {}
training_data["subj_queries"] = {}
training_data["obj_queries"] = {}
properties = {"P1001", "P106", "P1303", "P1376", "P1412", "P178", "P19", "P276", "P30", "P364", "P39", "P449", "P495", "P740", "P101", "P108", "P131", "P138", "P159", "P17", "P20", "P279", "P31", "P36", "P407", "P463", "P527", "P937", "P103", "P127", "P136", "P140", "P176", "P190", "P264", "P27", "P361", "P37", "P413", "P47", "P530"}
#wikidata_training_path = "/home/kalo/conferences/akbc2021/data/data/TREx/wikidata_training.json"
common_vocab_file = "/home/kalo/conferences/akbc2021/common_vocab_cased.txt"
wikidata_training_path = "/data/fichtel/BERTriple/training_datasets/wikidata41.json"
entity_dictionary_trex_label_file = "/data/fichtel/BERTriple/entity2label_trexlabel.json"
trex_triple_file = "/data/fichtel/BERTriple/test_datasets/"+"LAMA_trex_test.json"

with open(entity_dictionary_trex_label_file, "r") as f:
    entity_dictionary_trex_label = json.load(f)
    with open(common_vocab_file, "r") as f:
        common_vocab = set()
        for line in f:
            common_vocab.add(line.strip())
        with open("/data/wikidata/latest-truthy.nt", "r") as f:
            for i, line in enumerate(f):
                try:
                    s,p,o = line.split("> <")
                    if "wikidata.org/entity" in o and "wikidata.org/entity" in s:

                        s_qid = s.replace("<", "").replace("http://www.wikidata.org/entity/","")
                        o_qid = o.replace("> .\n","").replace("http://www.wikidata.org/entity/","")
                        p_qid = p.replace("http://www.wikidata.org/prop/direct/","").replace(">","")
                        if p_qid in properties:
                            if s_qid in entity_dictionary_trex_label and o_qid in entity_dictionary_trex_label:
                                s_label = entity_dictionary_trex_label[s_qid]
                                o_label = entity_dictionary_trex_label[o_qid]
                            
                            if o_label in common_vocab:
                                triple = {"subj": s_label, "prop": p_qid, "obj": o_label}
                                if p_qid not in training_data["obj_queries"]:
                                    training_data["obj_queries"][p_qid] = [triple]
                                else:
                                    training_data["obj_queries"][p_qid].append(triple)
                except ValueError:
                    continue


#remove duplicates of triples
training_data["obj_queries"] = remove_duplicates(training_data["obj_queries"])

with open(trex_triple_file, "r") as f:
    trex_test = json.load(f)
    #remove testdata from training data
    count_all_triples = 0
    for p_qid in training_data["obj_queries"]:
        for triple in training_data["obj_queries"][p_qid]:
            count_all_triples = count_all_triples + 1
    print("Lengths before substraction test from training data {}.".format(count_all_triples))
    training_data["obj_queries"] = substract_triples(training_data["obj_queries"], trex_test)
    count_all_triples = 0
    for p_qid in training_data["obj_queries"]:
        for triple in training_data["obj_queries"][p_qid]:
            count_all_triples = count_all_triples + 1
    print("Lengths after substraction test from training data {}.".format(count_all_triples))

    with open(wikidata_training_path, "w+") as f:
        json.dump(training_data, f, indent=4, sort_keys=True)

Lengths before substraction test from training data 106.
Lengths after substraction test from training data 106.


In [60]:
def check_triple(obj_label, tokenizer):
    tokenized_obj = tokenizer.tokenize(obj_label)
    if "[UNK]" not in tokenized_obj:
        if len(tokenized_obj) == 1:
            return True
    return False

with open("/data/kalo/iswc2020/LAMA/common_vocab_cased.txt", "r") as f:
    for line in f:
        if check_triple(line, tokenizer) == False:
            print(line)
        else:
            print(line)
            break


#



In [10]:
# counter for properties in wikidata training dataset
from collections import Counter
c = []
with open(wikidata_training_path, "r") as f:
    training_data = json.load(f)
    for d in training_data:
        c.append(d["predicate"])
        
counter = Counter(c)
print(counter.most_common)

<bound method Counter.most_common of Counter({'P31': 20508383, 'P407': 11317828, 'P17': 5201482, 'P106': 5053504, 'P27': 2221662, 'P1412': 1782630, 'P279': 1077521, 'P19': 828824, 'P131': 780222, 'P20': 412882, 'P495': 347826, 'P413': 312834, 'P136': 231264, 'P364': 228731, 'P937': 175686, 'P1303': 163219, 'P1001': 149718, 'P103': 148823, 'P276': 121847, 'P159': 114081, 'P140': 98585, 'P39': 58426, 'P101': 50890, 'P30': 49295, 'P527': 44526, 'P138': 19536, 'P361': 19473, 'P740': 18941, 'P264': 14681, 'P449': 10660, 'P37': 10621, 'P47': 7392, 'P176': 6186, 'P108': 6041, 'P127': 4884, 'P190': 4749, 'P530': 4195, 'P178': 3223, 'P36': 2500, 'P463': 1757, 'P1376': 337})>
